In [1]:
import csv
import json
import numpy as np
import pandas as pd
import country_converter as coco

In [2]:
fp = "results_nonbilateral.xlsx"

In [3]:
# Load the data into a DataFrame
df = pd.read_excel(fp, sheet_name="Sheet1")

In [4]:
# Filter for year 2022
df = df[df["Year"] == 2022]

In [5]:
# Add 'market' to Excludes
def add_market(x):
    items = [item.strip() for item in str(x).split(";") if item.strip()]
    if "market" not in items:
        items.append("market")
    return "; ".join(sorted(set(items)))

df["Excludes"] = df["Excludes"].apply(add_market)

In [6]:
country_map = {
    c: coco.convert(c, to="iso2") for c in df["Importing Country"].unique()
}

country_map.update(
    {c: coco.convert(c, to="iso2") for c in df["Exporting Country"].unique()}
)

for k, v in country_map.items():
    if v == "not found":
        print(k)

country_map = {k: v for k, v in country_map.items() if v != "not found"}

Other Asia, nes not found in regex
Global not found in regex
Other Asia, nes not found in regex
Unknown not found in regex


Other Asia, nes
Global
Unknown


In [7]:
data = []

df["Exporting Location"] = df["Exporting Country"].map(country_map).fillna("GLO")
df["Importing Location"] = df["Importing Country"].map(country_map).fillna("GLO")

grouped = df.groupby(["Resource Name", "Exporting Country", "Importing Country"])

for (resource, export_country, importing_country), group in grouped:
    cf_series = group["GeoPolRisk Characterization Factor Normalized to copper [-]"]
    if not cf_series.empty and cf_series.mean() != 0:
        supplier_info = {
            "name": group["Dataset name"].iloc[0],
            "reference product": group["Dataset reference product"].iloc[0],
            "location": group["Exporting Location"].iloc[0],
            "operator": group["Operator"].iloc[0],
            "matrix": "technosphere"
        }

        # Add excludes if available and non-null
        excludes_val = group["Excludes"].iloc[0]
        if pd.notna(excludes_val) and excludes_val != "":
            supplier_info["excludes"] = [e.strip() for e in excludes_val.split(";") if e.strip()]

        data.append({
            "supplier": supplier_info,
            "consumer": {
                "location": group["Importing Location"].iloc[0],
                "matrix": "technosphere"
            },
            "value": cf_series.mean(),
        })

In [8]:
# data = []

# df["Exporting Location"] = df["Exporting Country"].map(country_map).fillna("GLO")
# df["Importing Location"] = df["Importing Country"].map(country_map).fillna("GLO")


# grouped = df.groupby(["Resource Name", "Exporting Country", "Importing Country"])

# for (resource, export_country, importing_country), group in grouped:
#     if not group["GeoPolRisk Characterization Factor Normalized to copper [-]"].empty:
#         if group["GeoPolRisk Characterization Factor Normalized to copper [-]"].mean() != 0:
#             data.append({
#                 "supplier": {
#                     "name": group["Dataset name"].iloc[0],
#                     "reference product": group["Dataset reference product"].iloc[0],
#                     "location": group["Exporting Location"].iloc[0],
#                     "operator": group["operator"].iloc[0],
#                     "matrix": "technosphere"
#                 },
#                 "consumer": {
#                     "location": group["Importing Location"].iloc[0],
#                     "matrix": "technosphere"
#                 },
#                 "value": group["GeoPolRisk Characterization Factor Normalized to copper [-]"].mean(),
#             })


In [9]:
len(data)

43015

In [10]:
data[0]

{'supplier': {'name': 'aluminium production, primary',
  'reference product': 'aluminium, primary',
  'location': 'AL',
  'operator': 'startswith',
  'matrix': 'technosphere',
  'excludes': ['alloy', 'liquid', 'market']},
 'consumer': {'location': 'BA', 'matrix': 'technosphere'},
 'value': 0.0001192271411315043}

In [11]:
target_flow = [
    d for d in data
    if d["supplier"]["reference product"] == "aluminium, primary"
    and d["supplier"]["location"] == "AU"
    and d["consumer"]["location"] == "CA"
]
target_flow

[{'supplier': {'name': 'aluminium production, primary',
   'reference product': 'aluminium, primary',
   'location': 'AU',
   'operator': 'startswith',
   'matrix': 'technosphere',
   'excludes': ['alloy', 'liquid', 'market']},
  'consumer': {'location': 'CA', 'matrix': 'technosphere'},
  'value': 0.0002417527334203126}]

In [7]:
# data[0]

{'supplier': {'name': 'aluminium production, primary',
  'reference product': 'aluminium, primary',
  'location': 'AU',
  'operator': 'startswith',
  'matrix': 'technosphere'},
 'consumer': {'location': 'CA', 'matrix': 'technosphere'},
 'value': 1.101763446251004e-10}

In [12]:
fp_out = "GeoPolRisk_paired_2024.json"
with open(fp_out, "w") as f:
    json.dump(
            {
                "name": f"GeoPolRisk - country pairs",
                "unit": "kg copper-eq.",
                "version": "1.0",
                "exchanges": data,
            },
            f,
            indent=2
        )